In [8]:
%pip install whisper
%pip install pyaudio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyaudio
import whisper
import numpy as np

# Load the Whisper model
model = whisper.load_model("small.en")  

# Audio settings
CHUNK = 1024  # Number of audio frames per buffer
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Mono audio
RATE = 16000  # Sampling rate (must match Whisper's requirement)

# Initialize PyAudio
p = pyaudio.PyAudio()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.loa

In [ ]:
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(f"Device {i}: {p.get_device_info_by_index(i)['name']}")

Device 0: MacBook Pro Microphone
Device 1: MacBook Pro Speakers
Device 2: Microsoft Teams Audio
Device 3: ZoomAudioDevice


In [5]:
import queue


audio_queue = queue.Queue()
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    input_device_index=0,  
    frames_per_buffer=CHUNK
)

try:
  while True:
      audio_data = stream.read(CHUNK, exception_on_overflow=False)
      audio_queue.put(audio_data)
      
      # Process transcription after collecting enough audio
      if audio_queue.qsize() >= 2 * RATE // CHUNK:  # Process ~1 second of audio
          audio_buffer = b''.join([audio_queue.get() for _ in range(audio_queue.qsize())])
          audio_array = np.frombuffer(audio_buffer, dtype=np.int16).astype(np.float32) / 32768.0
          
          result = model.transcribe(audio_array, fp16=False)
          print(result["text"])
except:
    print("Stopping the transcription")
    stream.stop_stream()
    stream.close()
    p.terminate()

 Bye.

 It's fine.
 Twinkle.
 You are
 about
 time and
 Good sleep
 The little st-

 One, two, one.
 you are.


Stopping the transcription


In [ ]:
import queue
import numpy as np
import pyaudio
import whisper
import threading
import matplotlib.pyplot as plt
from IPython.display import clear_output
import wave
import datetime

class LiveTranscriber:
    def __init__(self, model_name="base.en", device="cpu"):
        # Audio parameters
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 16000
        self.CHUNK = 1024
        self.SILENCE_THRESHOLD = 0.01
        
        # Initialize components
        self.model = whisper.load_model(model_name).to(device)
        self.audio_queue = queue.Queue()
        self.is_running = False
        self.transcriptions = []
        
        # Setup PyAudio
        self.p = pyaudio.PyAudio()
        self._setup_stream()
        
    def _setup_stream(self):
        """Initialize audio stream with error handling"""
        try:
            self.stream = self.p.open(
                format=self.FORMAT,
                channels=self.CHANNELS,
                rate=self.RATE,
                input=True,
                input_device_index=0,
                frames_per_buffer=self.CHUNK
            )
        except OSError as e:
            print(f"Error setting up audio stream: {e}")
            self.p.terminate()
            raise
            
    def _process_audio(self):
        """Process audio buffer and transcribe"""
        while self.is_running:
            if self.audio_queue.qsize() >= 1.5*self.RATE // self.CHUNK:
                # Get audio buffer
                audio_buffer = b''.join([self.audio_queue.get() for _ in range(self.audio_queue.qsize())])
                audio_array = np.frombuffer(audio_buffer, dtype=np.float32)
                
                # Check if audio contains speech
                if np.abs(audio_array).mean() > self.SILENCE_THRESHOLD:
                    try:
                        result = self.model.transcribe(audio_array, fp16=False)
                        if result["text"].strip():
                            timestamp = datetime.datetime.now().strftime("%H:%M:%S")
                            self.transcriptions.append(f"[{timestamp}] {result['text']}")
                            
                            # Display last 5 transcriptions
                            clear_output(wait=True)
                            print("\nLast transcriptions:")
                            for t in self.transcriptions:
                                print(t)
                            
                    except Exception as e:
                        print(f"Transcription error: {e}")
                
    def start(self):
        """Start transcription"""
        self.is_running = True
        self.processor_thread = threading.Thread(target=self._process_audio)
        self.processor_thread.start()
        
        try:
            while self.is_running:
                audio_data = self.stream.read(self.CHUNK, exception_on_overflow=False)
                self.audio_queue.put(audio_data)
                
        except KeyboardInterrupt:
            self.stop()
            
    def stop(self):
        """Stop transcription and cleanup"""
        self.is_running = False
        if hasattr(self, 'processor_thread'):
            self.processor_thread.join()
        
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()
        
        # Save transcriptions
        with open("output/transcription_log.txt", "w") as f:
            f.write("\n".join(self.transcriptions))
            
# Usage
transcriber = LiveTranscriber(model_name="base.en")
try:
    transcriber.start()
except KeyboardInterrupt:
    transcriber.stop()


Last transcriptions:
[09:41:36]  Twinkle, twinkle,
[09:41:38]  The Heatha briefing results classed
[09:41:40]  ow
[09:41:41]  on what you...


In [21]:
from whisper.tokenizer import get_tokenizer
from rapidfuzz import fuzz, process

class LiveTranscriber:
    def __init__(self, model_name="base.en", device="cpu", similarity_threshold=80):
        # Audio parameters
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 16000
        self.CHUNK = 1024
        self.SILENCE_THRESHOLD = 0.01
        self.similarity_threshold = similarity_threshold  # Minimum similarity percentage

        # Initialize components
        self.model = whisper.load_model(model_name).to(device)
        self.tokenizer = get_tokenizer(model_name)  # Initialize tokenizer
        self.audio_queue = queue.Queue()
        self.is_running = False
        self.transcriptions = []
        
        # Prepare lyrics for matching
        lyrics = "Twinkle, twinkle, little star, How I wonder what you are! Up above the world so high, Like a diamond in the sky."
        self.lyrics_tokens = self.tokenizer.encode(lyrics)  # Tokenize lyrics
        
        # Setup PyAudio
        self.p = pyaudio.PyAudio()
        self._setup_stream()
        
    def _setup_stream(self):
        """Initialize audio stream with error handling"""
        try:
            self.stream = self.p.open(
                format=self.FORMAT,
                channels=self.CHANNELS,
                rate=self.RATE,
                input=True,
                input_device_index=0,
                frames_per_buffer=self.CHUNK
            )
        except OSError as e:
            print(f"Error setting up audio stream: {e}")
            self.p.terminate()
            raise
            
    def _process_audio(self):
        """Process audio buffer and transcribe"""
        while self.is_running:
            if self.audio_queue.qsize() >= 1.5 * self.RATE // self.CHUNK:
                # Get audio buffer
                audio_buffer = b''.join([self.audio_queue.get() for _ in range(self.audio_queue.qsize())])
                audio_array = np.frombuffer(audio_buffer, dtype=np.float32)
                
                # Check if audio contains speech
                if np.abs(audio_array).mean() > self.SILENCE_THRESHOLD:
                    try:
                        result = self.model.transcribe(audio_array, fp16=False)
                        if result["text"].strip():
                            transcript = result["text"]
                            matching_tokens = self._match_tokens(transcript)
                            
                            if matching_tokens:
                                timestamp = datetime.datetime.now().strftime("%H:%M:%S")
                                matched_text = self.tokenizer.decode(matching_tokens)
                                self.transcriptions.append(f"[{timestamp}] {matched_text}")
                                
                                # Display last 5 transcriptions
                                clear_output(wait=True)
                                print("\nLast transcriptions:")
                                for t in self.transcriptions[-5:]:
                                    print(t)
                            
                    except Exception as e:
                        print(f"Transcription error: {e}")
                        
    def _match_tokens(self, transcript):
        """Match tokens from the transcript to the lyrics with fuzzy matching"""
        # Tokenize the transcription text
        transcript_tokens = self.tokenizer.encode(transcript)
        matched_tokens = []

        for token in transcript_tokens:
            # Decode the token into a string
            token_text = self.tokenizer.decode([token]).strip()

            # Perform fuzzy matching on the string representation
            match = process.extractOne(
                token_text, [self.tokenizer.decode([lt]).strip() for lt in self.lyrics_tokens], scorer=fuzz.ratio
            )

            if match and match[1] >= self.similarity_threshold:  # Ensure similarity meets threshold
                matched_tokens.append(token)

        return matched_tokens if matched_tokens else None
                
    def start(self):
        """Start transcription"""
        self.is_running = True
        self.processor_thread = threading.Thread(target=self._process_audio)
        self.processor_thread.start()
        
        try:
            while self.is_running:
                audio_data = self.stream.read(self.CHUNK, exception_on_overflow=False)
                self.audio_queue.put(audio_data)
                
        except KeyboardInterrupt:
            self.stop()
            
    def stop(self):
        """Stop transcription and cleanup"""
        self.is_running = False
        if hasattr(self, 'processor_thread'):
            self.processor_thread.join()
        
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()
        
        # Save transcriptions
        with open("output/transcription_log.txt", "w") as f:
            f.write("\n".join(self.transcriptions))

transcriber = LiveTranscriber(model_name="base.en")
try:
    transcriber.start()
except KeyboardInterrupt:
    transcriber.stop()


Last transcriptions:
[17:25:56]  you
[17:25:58]  what
[17:25:59]  I.
[17:26:01]  the
[17:26:02]  are.


In [22]:
import requests

def get_currently_playing_track(access_token):
    url = "https://api.spotify.com/v1/me/player/currently-playing"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        track_name = data['item']['name']
        artist_name = data['item']['artists'][0]['name']
        return track_name, artist_name
    else:
        print("No track is currently playing or error occurred.")
        return None, None

In [23]:
def match_transcription_to_lyrics(transcription, lyrics, threshold=80):
    for line in lyrics.split("\n"):
        if fuzz.partial_ratio(transcription.lower(), line.lower()) >= threshold:
            return f"Match found in lyrics: {line}"
    return "No match found in lyrics."